In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from statsmodels.tsa.stattools import adfuller
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.transformations.series.summarize import WindowSummarizer
from sktime.forecasting.compose import make_reduction
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def lightgbm_forecast_lags(series, district_name, 
                          max_lags=6, 
                          window_features=None,
                          differencing=True,
                          use_cv=True,
                          custom_params=None):
    """
    LightGBM time series forecasting with proper temporal validation
    """
    # Create output directory
    os.makedirs('LightGBM_Forecasts', exist_ok=True)
    
    # 1. Stationarity handling
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:
            d = 1
            series = series.diff().dropna()

    # 2. Temporal split using sktime's proper method
    y_train, y_test = temporal_train_test_split(series, test_size=0.2)

    # 3. Create feature engineering pipeline
    features = []
    if max_lags > 0:
        features.append(("lags", WindowSummarizer(
            lag_feature={"lag": list(range(1, max_lags+1))},
            truncate="bfill"
        )))
    
    if window_features:
        features.append(("window", WindowSummarizer(
            lag_feature={
                "mean": [[1, window_features['mean_window']]],
                "std": [[1, window_features['std_window']]]
            },
            truncate="bfill"
        )))

    # 4. LightGBM regressor setup
    regressor = lgb.LGBMRegressor(
        **custom_params or {
            'objective': 'regression',
            'metric': 'rmse',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'n_estimators': 500,
            'random_state': 42
        }
    )
    
    # 5. Create forecasting pipeline
    forecaster = make_reduction(
        estimator=Pipeline([
            ("features", Pipeline(features)),
            ("regressor", regressor)
        ]),
        window_length=max_lags,
        strategy="recursive"
    )

    # 6. Hyperparameter tuning with temporal CV
    if use_cv:
        param_grid = {
            'estimator__regressor__num_leaves': [15, 31, 63],
            'estimator__regressor__learning_rate': [0.01, 0.05, 0.1],
            'estimator__regressor__min_child_samples': [10, 20, 50]
        }
        
        forecaster = GridSearchCV(
            forecaster,
            param_grid,
            cv=TimeSeriesSplit(n_splits=5),
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )

    # 7. Training
    forecaster.fit(y_train)
    
    # 8. Forecasting
    fh = np.arange(len(y_test)) + 1  # Forecast horizon
    y_pred = forecaster.predict(fh)
    
    # 9. Inverse differencing
    if d == 1:
        last_train_value = original_series.iloc[-len(y_test)-1]
        y_pred = pd.Series(np.cumsum(y_pred) + last_train_value, index=y_test.index)
        y_test = original_series.iloc[-len(y_test):]

    # 10. Create results dataframe
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast': y_pred.values
    })

    # 11. Save results
    forecast_df.to_csv(
        f'LightGBM_Forecasts/{district_name}_forecast.csv',
        index=False
    )

    # 12. Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    metrics = {
        'district': district_name,
        'rmse': rmse,
        'best_params': getattr(forecaster, 'best_params_', None),
        'differencing': d
    }

    # 13. Visualization
    plt.figure(figsize=(14, 7))
    plt.plot(original_series.index, original_series, label='Original')
    plt.plot(forecast_df['date'], forecast_df['forecast'], 
            label='LightGBM Forecast', alpha=0.7)
    plt.title(f'{district_name} Forecast\nRMSE: {rmse:.2f}')
    plt.legend()
    plt.savefig(f'LightGBM_Forecasts/{district_name}_forecast_plot.png')
    plt.close()

    return metrics, forecast_df

In [4]:
districts = data['district'].unique()
rmse_values = []

# Usage Example


def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I48"].asfreq('MS')
        
        results, forecast = lightgbm_forecast_lags(
        series=ts,
        district_name=district,
        max_lags=3,
        window_features={'mean_window': 3, 'std_window': 6},
        custom_params={
            'num_leaves': 63,
            'learning_rate': 0.1,
            'n_estimators': 1000
        }
    )

        
    
        # Show results
        print("=== Metrics ===")
        print(results)
        print("\n=== Forecast Data ===")
        print(forecast)
    
    return results
run_for_each_district()

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan]
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000087 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76
[LightGBM] [Info] Number of data points in the train set: 35, number of used features: 6
[LightGBM] [Info] Start training from score 470.600000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan]
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000205 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 73
[LightGBM] [Info] Number of data points in the train set: 35, number of used features: 6
[LightGBM] [Info] Start training from score 455.971429
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan]
  warnings.warn(


...

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan]
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000156 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 74
[LightGBM] [Info] Number of data points in the train set: 35, number of used features: 6
[LightGBM] [Info] Start training from score 215.028571
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
27 fits failed out of a total of 135.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
27 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 893, in _fit_and_score
    estimator.fit(X_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sktime\forecasting\base\_base.py", line 396, in fit
    self._fit(y=y_inner, X=X_inner, fh=fh)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\s

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 6, number of used features: 0
[LightGBM] [Info] Start training from score 838.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the spl

{'district': 'MUMBAI SUBURBAN',
 'rmse': 217.07295240694236,
 'best_params': {'estimator__regressor__learning_rate': 0.01,
  'estimator__regressor__min_child_samples': 10,
  'estimator__regressor__num_leaves': 15},
 'differencing': 0}

In [5]:
# !pip install "dask<2025.1"
